In [30]:
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC
from datasets import load_dataset
import torch

import soundfile as sf


 
 # load model and processor
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-lv-60-espeak-cv-ft")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-lv-60-espeak-cv-ft")
     
#  # load dummy dataset and read soundfiles
# ds = load_dataset("patrickvonplaten/librispeech_asr_dummy", "clean", split="validation")
# print(ds[0]["audio"]["array"])
# print(ds[0]['text'])


In [42]:
def map_to_array(batch):
    speech_array, _ = sf.read(batch)
    return speech_array
data = map_to_array('audio/output.flac')  
 # tokenize
input_values = processor(data, return_tensors="pt").input_values
 
 # retrieve logits
with torch.no_grad():
    logits = model(input_values).logits
 
 # take argmax and decode
predicted_ids = torch.argmax(logits, dim=-1)
transcription = processor.batch_decode(predicted_ids)
 # => should give ['m ɪ s t ɚ k w ɪ l t ɚ ɹ ɪ z ð ɪ ɐ p ɑː s əl ʌ v ð ə m ɪ d əl k l æ s ᵻ z æ n d w iː ɑːɹ ɡ l æ d t ə w ɛ l k ə m h ɪ z ɡ ɑː s p əl']
print(transcription)

# data_s = map_to_array('audio/test2monostutter.flac')  
#  # tokenize
# input_values = processor(data_s, return_tensors="pt").input_values
 
#  # retrieve logits
# with torch.no_grad():
#     logits = model(input_values).logits
 
#  # take argmax and decode
# predicted_ids = torch.argmax(logits, dim=-1)
# transcriptionStutter = processor.batch_decode(predicted_ids)
#  # => should give ['m ɪ s t ɚ k w ɪ l t ɚ ɹ ɪ z ð ɪ ɐ p ɑː s əl ʌ v ð ə m ɪ d əl k l æ s ᵻ z æ n d w iː ɑːɹ ɡ l æ d t ə w ɛ l k ə m h ɪ z ɡ ɑː s p əl']
# print(transcriptionStutter)
transcriptionStutter = ["sɪmpli pʊt, ə ˈpɛrəˌgræf ɪz ə kəˈlɛkʃən əv ˈsɛntənsɪz ɔl rɪˈleɪtɪd tɪ ə ˈsɛntrəl ˈtɑpɪk, aɪˈdiə, ər θim. ˈpɛrəˌgræfs ækt ɛz ˈstrəkʧərəl tulz fər ˈraɪtərz tɪ ˈɔrgəˌnaɪz ðɛr θɔts ˈɪntu ən aɪˈdil prəˈgrɛʃən, ənd ðeɪ ˈɔlsoʊ hɛlp ˈridərz ˈprɔˌsɛs ðoʊz θɔts ˈɛfərtləsli"]

It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


['s ɪ m p l i p ʊ ɾ ʌ p æ ɹ ɪ ɡ ɹ æ f æ ɛ k ɚ ɪ l ɛ k ʃ ə n ʌ v z æ n d t ə n s ᵻ z ɑːɹ ɹ ᵻ l eɪ ɾ ᵻ d t uː s ɛ n t ɹ əl t ɑː p ɪ k d iː ɜː θ iː m p æ ɹ ᵻ ɡ ɹ æ f s æ k s æ k t æ z s t ɹ ʌ k tʃ ɹ oʊ t ɔːɹ z f ɔːɹ ɹ aɪ ɾ ɚ z t ə l ᵻ n aɪ z ð ɚ θ ɑː t s ɪ n d t uː ɚ n aɪ d iː p ɹ oʊ ɡ ɹ ɛ ʃ ə n æ n d ð eɪ ɔː l s oʊ h ɛ l p ɹ iː d ɚ z p ɹ ə p ɹ ɑː s ɛ s ð oʊ z θ ɑː t s æ v ɛ f ɚ t l ɪ s t i']


In [45]:
print(transcriptionStutter)
searched = set()
stutterPhonemes = []
for phoneme in transcription[0]:
    if phoneme not in searched:
        searched.add(phoneme)
        og = transcription[0].count(phoneme)
        vc = transcriptionStutter[0].count(phoneme)
        score = float(vc) / float(og)
        print(score)
        if score > 2:
            stutterPhonemes.append(phoneme)




['sɪmpli pʊt, ə ˈpɛrəˌgræf ɪz ə kəˈlɛkʃən əv ˈsɛntənsɪz ɔl rɪˈleɪtɪd tɪ ə ˈsɛntrəl ˈtɑpɪk, aɪˈdiə, ər θim. ˈpɛrəˌgræfs ækt ɛz ˈstrəkʧərəl tulz fər ˈraɪtərz tɪ ˈɔrgəˌnaɪz ðɛr θɔts ˈɪntu ən aɪˈdil prəˈgrɛʃən, ənd ðeɪ ˈɔlsoʊ hɛlp ˈridərz ˈprɔˌsɛs ðoʊz θɔts ˈɛfərtləsli']
1.0
0.22905027932960895
1.3636363636363635
1.0
0.8888888888888888
1.5
1.0
0.6
0.0
0.0
0.2727272727272727
0.0
0.0
1.0
1.5714285714285714
0.8333333333333334
0.0
0.6666666666666666
4.166666666666667
1.0
0.5
1.0
0.7142857142857143
1.0714285714285714
0.0
0.2
0.0
1.0
1.0
0.0
1.0
0.5
2.0
1.3333333333333333
1.0
1.0


In [46]:
print(stutterPhonemes)

['ə']


In [16]:
# print(transcription)
# transcription = transcription[0].split(" ")
# transcriptionStutter = transcriptionStutter[0].split(" ")

# hello = transcription[:4]
# helloStutter = transcriptionStutter[:10]
# print(helloStutter, hello)
# for i in range(max(len(hello), len(helloStutter))):
#     if i >= len(hello):
#         print(" ", helloStutter[i])
#     elif i >= len(helloStutter):
#         print(hello[i], ' ')
#     else:
#         print(hello[i], helloStutter[i])


['h ʌ l oʊ m aɪ n eɪ m ɪ z h ɛ n ɹ i']
['x', 'a', 'ɑ', 'h', 'ɑ', 'x', 'x', 'ɑ', 'l', 'oː'] ['h', 'ʌ', 'l', 'oʊ']
h x
ʌ a
l ɑ
oʊ h
  ɑ
  x
  x
  ɑ
  l
  oː


In [29]:
from torch import le


col = {'x':'h', 'h':'h'}
vowel = {'i', 'e', }
testa = ['h', 'ʌ', 'l', 'oʊ']
testb = ['h', 'h', 'h', 'ʌ', 'l', 'oʊ', 'oʊ']
indexStutter = 0
stutterPho = []
for index, letter in enumerate(testa):
    if indexStutter >= len(testb):
        break
    if testb[indexStutter] != letter:
        stutterPho.append(letter)
        # if testb[indexStutter + 1] == letter:
    else:
        indexStutter += 1
        if indexStutter >= len(testb):
            break
        if index == len(testa) - 1:
            if testb[indexStutter] == letter:
                stutterPho.append(letter)
                break
        elif testb[indexStutter] == letter and (testa[index + 1] != letter):
            stutterPho.append(letter)
            indexStutter += 1
            if indexStutter >= len(testb):
                break
            while testb[indexStutter] == letter:
                indexStutter += 1
print(stutterPho)

['h', 'oʊ']
